In [1]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
#TEAM NAME: HURICANE 
#SEVKET UMUT GUNDUZ , TOYGUN OZYUREK

In [4]:
# IMPORT LIBRARIES 
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [5]:
# READ CSV FILES 
test = pd.read_csv('/kaggle/input/cmp4336-task-0/test.csv')
train = pd.read_csv('/kaggle/input/cmp4336-task-0/train.csv')

In [6]:
test.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [7]:
def binary_encode(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df 

def ordinal_encode(df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: ordering.index(x) if x in ordering else -1)
    return df 

def label_encode(df, columns):
    df = df.copy()
    label_encoder = LabelEncoder()
    for column in columns:
        df[column] = label_encoder.fit_transform(df[column])
    return df

In [8]:
test.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [9]:
def preprocess_inputs(train, test=None, pca_components=None):
    df = pd.concat([train, test], ignore_index=True) if test is not None else train.copy()

    # Categorize features
    binary_features = ['surgery', 'age', 'surgical_lesion', 'cp_data']
    positive_values = ['yes', 'adult', 'yes', 'yes']

    ordinal_features = ['temp_of_extremities', 'peripheral_pulse', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces']
    orderings = [
        ['cold', 'cool', 'normal', 'warm'],
        ['absent', 'reduced', 'normal', 'increased'],
        ['less_3_sec', '3', 'more_3_sec'],
        ['mild_pain', 'depressed', 'extreme_pain', 'severe_pain', 'alert', 'slight'],
        ['hypomotile', 'absent', 'hypermotile', 'normal', 'distend_small', 'moderate'],  # Added 'moderate'
        ['none', 'slight', 'moderate', '', 'severe'],
        ['none', 'slight', 'significant'],
        ['none', 'less_1_liter', 'more_1_liter', 'slight'],
        ['normal', 'decreased', 'absent', 'increased', 'serosanguious']
    ]

    nominal_features = ['hospital_number', 'mucous_membrane', 'abdomen', 'abdomo_appearance']
    
    for column in df.columns:
        if column in df.select_dtypes('object').columns:
            if column not in nominal_features:
                df[column] = df[column].fillna(df[column].mode()[0])
        else:
            df[column] = df[column].fillna(df[column].mean())

    # Encode categorical feature columns
    df = binary_encode(df, columns=binary_features, positive_values=positive_values)
    df = ordinal_encode(df, columns=ordinal_features, orderings=orderings)
    df = label_encode(df, columns=nominal_features)

    # Separate the combined dataset back into train and test
    if test is not None:
        train = df.iloc[:len(train)]
        test = df.iloc[len(train):].reset_index(drop=True)

    # Encode labels
    if 'outcome' in df.columns:
        label_mapping = {'lived': 0, 'died': 1, 'euthanized': 2}
        df['outcome'] = df['outcome'].replace(label_mapping)

        y = df['outcome'].copy()
        X = df.drop('outcome', axis=1).copy()

        # Standardize features using StandardScaler
        scaler = StandardScaler()
        X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
        
        if pca_components:
            pca = PCA(n_components=pca_components)
            X_pca = pd.DataFrame(pca.fit_transform(X), columns=[f'pca_{i+1}' for i in range(pca_components)])
            X = pd.concat([X, X_pca], axis=1)

        if test is not None:
            # Apply the same transformation to the test set
            test = pd.DataFrame(scaler.transform(test), columns=test.columns)
            if pca_components:
                test_pca = pd.DataFrame(pca.transform(test), columns=[f'pca_{i+1}' for i in range(pca_components)])
                test = pd.concat([test, test_pca], axis=1)

            return X, y, test
        else:
            return X, y
    else:
        # Standardize features using StandardScaler
        scaler = StandardScaler()
        X = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

        # Apply PCA if specified
        if pca_components:
            pca = PCA(n_components=pca_components)
            X_pca = pd.DataFrame(pca.fit_transform(X), columns=[f'pca_{i+1}' for i in range(pca_components)])
            X = pd.concat([X, X_pca], axis=1)

        if test is not None:
            # Apply the same transformation to the test set
            test = pd.DataFrame(scaler.transform(test), columns=test.columns)
            if pca_components:
                test_pca = pd.DataFrame(pca.transform(test), columns=[f'pca_{i+1}' for i in range(pca_components)])
                test = pd.concat([test, test_pca], axis=1)

            return X, test
        else:
            return X

In [10]:
X, y = preprocess_inputs(train, pca_components=15)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
k_values = [3, 5, 60]
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for k={k}: {accuracy:.2f}')

Accuracy for k=3: 0.58
Accuracy for k=5: 0.62
Accuracy for k=60: 0.69


In [13]:
# Assuming you have the preprocess_inputs function and test dataset
test_processed = preprocess_inputs(test, pca_components=15)

# Standardize features using the same scaler used for training data
X_test_scaled = scaler.transform(test_processed)

best_k = 60 # Replace with the best value of k from the previous step
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)


# Map numerical labels back to class names if necessary
label_mapping_reverse = {0: 'lived', 1: 'died', 2: 'euthanized'}
y_pred_names = [label_mapping_reverse[label] for label in y_pred]

In [14]:
# Create a result DataFrame with indices
result_df = pd.DataFrame({'outcome': y_pred_names})
if 'id' in test.columns:
    result_df['id'] = test['id']
result_df.set_index('id', inplace=True)

# Save the result DataFrame to a CSV file
result_df.to_csv('final.csv')

# Display the result DataFrame
print(result_df)

         outcome
id              
1235       lived
1236        died
1237       lived
1238  euthanized
1239       lived
...          ...
2054        died
2055  euthanized
2056        died
2057       lived
2058       lived

[824 rows x 1 columns]
